In [ ]:
from operator import itemgetter
import numpy as np
import holoviews as hv
import funcs
hv.notebook_extension('matplotlib', 'bokeh')
import adaptive
%opts Image {+axiswise} [colorbar=True aspect='square'] (cmap='viridis')

![](https://upload.wikimedia.org/wikipedia/commons/thumb/4/4f/3D_Spherical.svg/208px-3D_Spherical.svg.png)

# Chemical potential sweep

In [ ]:
import cloudpickle
with open('data/learner.cloudpickle', 'rb') as f:
    learner = cloudpickle.load(f)

In [ ]:
plots = {tuple(l.cdims.values()): l.plot().redim(x='V_bias', y='B') for l in learner.learners}
kdims = list(learner.learners[0].cdims.keys())
hm = hv.HoloMap(plots, kdims=kdims)

### _Note: in the low chemical potential regime ($\mu=10$ meV) the lines are so thin that they are not found._ 

In [ ]:
%%opts Image {+framewise}
%%output size=200
# hm
hv.util.Dynamic(hm)

# Sweep chemical potential 2 (fixed barrier)

In [ ]:
vals = funcs.named_product(g=[0, 50],
                           alpha=[0, 20],
                           orbital=[False, True],
                           direction=list('xyz'),
                           mu=list(range(10, 21)))

learners = []
for val in vals:
    f = lambda x: x
    learner = adaptive.Learner2D(f, [(-1, 1), (0, 2)])
    learner.cdims = val
    learners.append(learner)

learner = adaptive.DataSaver(adaptive.BalancingLearner(learners), itemgetter('G_01'))
folder = 'tmp-2017-12-06-07h39m00s/'
funcs.load_DataSaver_extra_data(learner, folder=folder)
funcs.load_BalancingLearner_data(learners, folder=folder)

In [ ]:
plots = {tuple(l.cdims.values()): l.plot().redim(x='V_bias', y='B') for l in learner.learner.learners}
kdims = list(learner.learner.learners[0].cdims.keys())
hm = hv.HoloMap(plots, kdims=kdims)

In [ ]:
%%opts Image {+framewise}
%%output size=200
# hv.util.Dynamic(hm)
hm

# Electrical field sweep at fixed $\mu$
* fix tunnel barrier to not be dependent on $\mu$
* sweep E-field
* set $\mu$ in the SC to 100 meV (retune the gap for each potential)

In [ ]:
gaps = {-10: {0: 201.375, 90: 125.875},
        -8: {0: 178.375, 90: 116.625},
        -6: {0: 152.875, 90: 110.125},
        -4: {0: 129.625, 90: 103.125},
        -2: {0: 107.125, 90: 97.375},
         0: {0: 85.875, 90: 94.875},
         2: {0: 65.625, 90: 93.625},
         4: {0: 46.875, 90: 94.625},
         6: {0: 30.125, 90: 97.375}}

vals = funcs.named_product(g=[0, 50],
                           alpha=[0, 20],
                           orbital=[False, True],
                           direction=list('xyz'),
                           gradient=list(gaps.keys()),
                           angle=[0, 90])

learners = []
for val in vals:
    f = lambda x: 0
    learner = adaptive.Learner2D(f, [(-1, 1), (0, 2)])
    learner.cdims = val
    learners.append(learner)

learner = adaptive.BalancingLearner(learners)
folder = 'data/gradient-sweep-angle-0-90'
funcs.load_BalancingLearner_data(learners, folder)

In [ ]:
plots = {tuple(l.cdims.values()): l.plot().redim(x='V_bias', y='B') for l in learner.learners}
kdims = list(learner.learners[0].cdims.keys())
hm = hv.HoloMap(plots, kdims=kdims)

In [ ]:
%%opts Image {+framewise}
%%output size=200
hv.util.Dynamic(hm.select(B=(0, 2)))